# ASTR 244W: Observation Notebook
The purpose of this notebook is to assist during observational time. This includes calculating how many observations to obtain a predetermiend S/N based on the current seeing, a built-in center-me, as well as any other tools used. 

In [4]:
import numpy as np
import matplotlib.pyplot
from astropy.table import QTable
import astropy.units as u
from astropy.constants import h, e, c


In [57]:
# Parameters of the Mees Telescope -- the Filters

# Background noise per pixel per exposure time
n_per_time = (140 / 600 / u.s ) * 1.35 


filt = np.array(["B", "Hbeta", "[OIII]", "G", "R", "Halpha", "[SII]"])
wavelength = np.array([442, 486.1, 500.7, 521, 633, 656.3, 670.0]) * u.nm
bandwidth = np.array([128, 8.5, 8.5, 80.5, 101, 7.0, 8.0]) * u.nm
flux_density = np.array([6.18e-8, 1.76e-8, 4.67e-8, 4.10e-8, 2.32e-8, 1.76e-8, 1.92e-8])*u.erg/u.s/u.cm/u.cm/u.nm
flux = np.array([(bandwidth[0]*flux_density[0]).value, 9.84e-8, 4.22e-7,
                 (bandwidth[3]*flux_density[3]).value, (bandwidth[4]*flux_density[4]).value, 
                 1.31e-7, 1.63e-7]) * u.erg / u.s / u.cm / u.cm
quantum_efficiency = np.array([0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45])
transmission_of_optical_system = np.array([0.95, 0.75, 0.75, 0.95, 0.95, 0.75, 0.75])

filters_information = QTable([filt, wavelength, bandwidth, flux_density, flux, quantum_efficiency, transmission_of_optical_system],
                  names = ('Filter', 'Wavelength', 'Bandwidth', 'Flux Density', 'Flux', 'Quantum Efficiency', 'Transmission of Optical System'),
                  meta= {'name': 'Filter parameters'})

pix_per_arc = (0.224 *u.arcsec )
collecting_area = 2700 * u.cm * u.cm

filters_information 

Filter,Wavelength,Bandwidth,Flux Density,Flux,Quantum Efficiency,Transmission of Optical System
,nm,nm,erg / (cm2 nm s),erg / (cm2 s),,
str6,float64,float64,float64,float64,float64,float64
B,442.0,128.0,6.18e-08,7.9104e-06,0.45,0.95
Hbeta,486.1,8.5,1.76e-08,9.84e-08,0.45,0.75
[OIII],500.7,8.5,4.67e-08,4.22e-07,0.45,0.75
G,521.0,80.5,4.1e-08,3.3005000000000004e-06,0.45,0.95
R,633.0,101.0,2.32e-08,2.3432e-06,0.45,0.95
Halpha,656.3,7.0,1.76e-08,1.31e-07,0.45,0.75
[SII],670.0,8.0,1.92e-08,1.63e-07,0.45,0.75


In [94]:
# Number of Pixels affected by seeing
def seeing_to_pixels(seeing):
    solid_angle = np.pi * seeing**2 / 4.0
    return np.ceil(solid_angle / pix_per_arc**2 + 1)

# Determine intensity of background light per electron charge
def I_B(N_pixels, t_exposure):
    result =  np.sqrt(N_pixels * (n_per_time * t_exposure)) / t_exposure
    return result

# Power needed to acheive a given SNR 
def P_SNR(desired_snr, seeing_pixel_num, t_exposure, tau, nu, lambda0):
    return (
        (desired_snr * h * c *  I_B(seeing_pixel_num, t_exposure) )
            / (tau * nu  * lambda0 )
            ).to(u.W)

def power_of_target(magnitude, filter):
    # Check if the filter is in the table
    if filter not in filters_information['Filter']:
        print(f'{filter} not in filters. Please choose from {filters_information["Filter"]}')
        return None
    loc = np.where(filters_information['Filter'] == filter)[0][0]


    F_mag = (filters_information[loc]['Flux Density'] * filters_information[loc]['Bandwidth'] * 10**(-magnitude/2.5)) * collecting_area
    return F_mag.to(u.W)

# Number of Observations of a target with a given magnitude in a given filter
def num_obs(magnitude, filter, desired_snr, seeing, t_exposure):
    seeing_pixel_num = seeing_to_pixels(seeing)
    power = power_of_target(magnitude, filter)
    if power is None:
        return None
    
    # Take lambda0 to be the wavelength of the filter
    lambda0 = filters_information[filters_information['Filter'] == filter]['Wavelength'][0]
    tau = filters_information[filters_information['Filter'] == filter]['Transmission of Optical System'][0]
    nu = filters_information[filters_information['Filter'] == filter]['Quantum Efficiency'][0]    

    P_desired = P_SNR(desired_snr, 
              seeing_pixel_num, 
              t_exposure,
              tau, 
              nu,
              lambda0)
    
    return P_desired / power 

def print_table(num_obs_tab, snr, filter, t_exposure):
    for i, v in enumerate(filter):
        print(f'For {v} filter: A total observation time of {num_obs_tab[:,i]} are needed for SNR of {snr}.')
    
# Determine the number of observations given various parameters
def num_obs_table(magnitude, seeing, t_exposure):
    snr = np.array([5, 10])
    filter = filters_information['Filter']
    num_obs_tab = np.zeros((len(snr), len(filters_information)))
    for i in range(len(snr)):
        for j in range(len(filter)):
            num_obs_tab[i][j] = num_obs(magnitude, filter[j], snr[i], seeing, t_exposure)
    num_obs_tab = num_obs_tab **2 * t_exposure
    print_table(num_obs_tab, snr, filter, t_exposure)
    

In [99]:
# Examples completed in class
# 10th magnitude star in G filter
Example1 = power_of_target(10, 'G')
print(f'Power of 24th magnitude target in G filter: {Example1}')

# Number of Pixels effected by seeing of 2 arcsec
Example2 = seeing_to_pixels(2.0 * u.arcsec)
print(f'Number of pixels affected by seeing of 2 arcsec: {Example2}')

# Power needed to acheive S/N = 5
Example2Half = P_SNR(5, Example2, 600*u.s, 0.95, 0.45, 0.47 * u.um)

print(f'Power needed to acheive S/N = 5: {Example2Half}')

# Number of 600 second observations of 24th magnitude star in G filter to acheive S/N = 5
Example3 = num_obs(magnitude = 24,
                   filter = 'G',
                   desired_snr = 5,
                   seeing = 2.0 * u.arcsec,
                   t_exposure = 600*u.s)

print(f'f={Example3}')
print(f'Number of {600*u.s} exposures: {Example3**2}')

print(f'Total observation time: {Example3**2 * 600 * u.s}')
print("--------------------------------------------")
print("--------------------------------------------\n")
print("For a 9.52 magnitude star:")
print("__________________________")
num_obs_table(magnitude = 20, seeing = 5.0 * u.arcsec, t_exposure = 30 * u.s)

Power of 24th magnitude target in G filter: 8.911350000000002e-14 W
Number of pixels affected by seeing of 2 arcsec: 64.0
Power needed to acheive S/N = 5: 9.061130774484412e-19 W
f=3.651733131342112
Number of 600.0 s exposures: 13.335154862541666
Total observation time: 8001.092917524999 s
--------------------------------------------
--------------------------------------------

For a 9.52 magnitude star:
__________________________
For B filter: A total observation time of [ 7.49817173 29.99268693] s are needed for SNR of [ 5 10].
For Hbeta filter: A total observation time of [ 27810.38913224 111241.55652898] s are needed for SNR of [ 5 10].
For [OIII] filter: A total observation time of [ 3723.01425895 14892.05703582] s are needed for SNR of [ 5 10].
For G filter: A total observation time of [ 31.00001403 124.00005611] s are needed for SNR of [ 5 10].
For R filter: A total observation time of [ 41.66493165 166.65972661] s are needed for SNR of [ 5 10].
For Halpha filter: A total obser

# Plot 

In [9]:
# Plot S/N as a function of total observation time
